In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq, least_squares
import matplotlib.pylab as plt
import math


In [2]:
def impliedVolatility(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalCall(S, K, r, x, T),
                                1e-12, 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalPut(S, K, r, x, T),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

def bachelierImpliedVolatility(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                BachelierCall(S, K, T, r, x),
                                1e-12, 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                BachelierPut(S, K, T, r, x),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

def preprocess_options_data(file_path):
    """
    Preprocess options data by calculating mid-price, scaling strike prices, 
    and mapping call/put flags.

    Parameters:
    - file_path (str): Path to the CSV file containing options data.

    Returns:
    - DataFrame: Processed options data with 'mid', 'strike', and 'payoff' columns.
    """
    df = pd.read_csv(file_path)
    df['mid'] = 0.5 * (df['best_bid'] + df['best_offer'])
    df['strike'] = df['strike_price'] * 0.001  # Adjusting strike price scale
    df['payoff'] = df['cp_flag'].map(lambda x: 'call' if x == 'C' else 'put')
    return df

def BlackScholesLognormalCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesLognormalPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

def BachelierCall(S0, K, T, r, sig)->tuple:
    """
    Calculate the discounted Bachelier prices of European call options.
    """
    d = (S0 - K)/(sig*T**.5)
    
    return math.exp(-r*T) * ( (S0 - K)*norm.cdf( d) + sig*T**.5*norm.pdf( d))

def BachelierPut(S0, K, T, r, sig)->tuple:
    """
    Calculate the discounted Bachelier prices of European put options.
    """
    d = (S0 - K)/(sig*T**.5)
    
    return math.exp(-r*T) * (-(S0 - K)*norm.cdf(-d) + sig*T**.5*norm.pdf(-d))


In [3]:

df_rates = pd.read_csv("zero_rates_20201201.csv")\
            .sort_values('days')
df_spx = preprocess_options_data("SPX_options.csv")
df_spy = preprocess_options_data("SPY_options.csv")

S_spx = 3662.45
S_spy = 366.02
exdate = 20210115
days_to_expiry = (pd.Timestamp(str(exdate)) - pd.Timestamp('2020-12-01')).days
r = np.interp(days_to_expiry, df_rates['days'].values, df_rates['rate'].values)
T = days_to_expiry / 365


# QF620 Project Part 3
Suppose on 1-Dec-2020, we need to evaluate an exotic European derivative
expiring on 15-Jan-2021
## 1. Contract Payoff function
Payoff function is

$$
S^{\frac{1}{3}}_T + 1.5 \times \log(S_T) + 10.0

$$


### 1.1 Black-Scholes Model
under Black-Scholes

$$
S_T = S_0 e^{(r - \frac{\sigma^2}{2})T + \sigma W_T}

$$

Therefore

$$
\begin{aligned}

V_0 
&= e^{-rT} \mathbb{E}
\left[
    {S_0 e^{\frac{1}{3} (r - \frac{\sigma^2}{2})T + \frac{1}{3} \sigma W_T}}
    + 1.5 \times \log(S_0 e^{(r - \frac{\sigma^2}{2})T 
    + \sigma W_T}) + 10.0
\right] \\
&= 
e^{-rT}
\left[
{S_0 e^{\frac{1}{3} (r - \frac{\sigma^2}{2})T}} \mathbb{E}[{e^{\frac{1}{3} \sigma W_T}}] +

	 1.5 \left[ 
		 \log{S_0} + \left(r - \frac{\sigma^2}{2} \right) T  + \mathbb{E}[\sigma W_T] 
	\right]
	+
	10.0
\right]
\\\\
V_0
&= 
e^{-rT}
\left[
{S_0 e^{\frac{1}{3} (r - \frac{\sigma^2}{3})T}} +

	 1.5 \left[ 
		 \log{S_0} + \left(r - \frac{\sigma^2}{2} \right) T 
	\right]
	+
	10.0
\right]
\\

\end{aligned}
$$


#### 1.1.1 $\sigma$ used
There is many market implied volatilities 
We can try using ATM strike volatility to price as a risk neutral measure where $K = S_0 e^{rT}$



##### 1.1.1.1 SPX 

In [4]:
F = S_spx * np.exp(r * T)
# Filter for the current expiration date
df_ex = df_spx[df_spx["exdate"] == exdate].copy()

# Filter and collect strikes and implied vols
df_ex['vols'] = df_ex.apply(lambda x: impliedVolatility(S_spx, x['strike'], r, x['mid'], T, x['payoff']), axis=1)
df_ex.dropna(inplace=True)
call_df, put_df = df_ex[df_ex['payoff'] == 'call'], df_ex[df_ex['payoff'] == 'put']
strikes = put_df['strike'].values
impliedvols = [
    call_df[call_df['strike'] == K]['vols'].values[0] 
        if K > S_spx else put_df[put_df['strike'] == K]['vols'].values[0]
            for K in strikes
]
df_iv = pd.DataFrame({'strike': strikes, 'impliedvol': impliedvols})
closest_strike = df_iv['strike'].iloc[(df_iv['strike'] - S_spx).abs().idxmin()]

# get pricing vol for contract
spx_ATM_implied_vol = df_iv[df_iv['strike'] == closest_strike]['impliedvol'].values[0]


#### 1.1.2 Valuation


In [5]:
np.exp(-r * T) *\
    (
        S_spx * np.exp(1/3 * (r - (spx_ATM_implied_vol**2) / 3) * T) \
        + 1.5 * (np.log(S_spx) + (r - (spx_ATM_implied_vol**2) / 2) * T)
        + 10
	)


3619.3971131390895


### 1.2 Bachelier Model

$$
dS_T = \mu dt + \sigma dW_t
$$
$$
S_T = S_0 + \mu T + \sigma W_T
$$


#### 1.2.1 $\sigma$ used


In [6]:
ATM_put = df_ex[(df_ex['strike'] == closest_strike) & (df_ex['payoff'] == 'put')]

In [7]:
ATM_put

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,mid,strike,payoff,vols
1478,20201201,20210115,P,3660000,94.8,95.5,E,95.15,3660.0,put,0.270597


In [ ]:
bachelier_sigma = brentq(lambda x: 95.15 - BachelierCall(S_spx, 3660, T, r, x),
                                1e-12, 1000)


#### 1.2.2 Valuation


In [9]:
import numpy as np

# Parameters
S0 = S_spx        # Initial stock price
sigma = bachelier_sigma     # Volatility in the Bachelier model
num_simulations = 100000  # Number of Monte Carlo simulations

# Simulate end prices S_T under Bachelier model
# S_T = S0 + (r * T) + sigma * sqrt(T) * Z
Z = np.random.normal(0, 1, num_simulations)
S_T = S0 + r * T + sigma * np.sqrt(T) * Z

# Calculate payoff for each simulated S_T
payoffs = S_T**(1/3) + 1.5 * np.log(S_T) + 10.0

# Discount the expected payoff
discounted_payoff = np.exp(-r * T) * np.mean(payoffs)

print(f"The estimated price of the derivative contract is: {discounted_payoff:.4f}")

The estimated price of the derivative contract is: 36.7698



### 1.2 Static Replication of European Payoff Model
*using SABR model calibrated from Part 2*
#### 1.2.1 Calibrated SABR model
SPX: Exdate 20210115: alpha = 1.817, rho = -0.404, nu = 2.790

SPY: Exdate 20210115: alpha = 0.908, rho = -0.489, nu = 2.729
#### 1.2.2 Valuation

## 2. Variance Swap
### 2.1 Black-Scholes Model
#### 2.1.1 $\sigma$ used
#### 2.1.2 Valuation

### 2.2 Bachelier Model
#### 2.2.1 $\sigma$ used
#### 2.2.2 Valuation

### 2.2 Static Replication of European Payoff Model
*using SABR model calibrated from Part 2*
#### 2.2.1 Calibrated SABR model
#### 2.2.2 Valuation